<div><img src="https://storage.googleapis.com/kaggle-competitions/kaggle/9120/logos/header.png?t=2018-04-02-23-51-59"></img></div>

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/9120/logos/thumb76_76.png?t=2018-04-02-23-45-04" align="left" width = "100px"/>

<h1> Home Credit Default Risk Step by Step: 2nd Notebook</h1>

<div style="clear:both"></div>

<br>

[Home Credit Default Risk Step by Step: 1st Notebook](https://www.kaggle.com/ekrembayar/homecredit-default-risk-step-by-step-1st-notebook)


# 1. PACKAGES

In [2]:
# 1. PACKAGES
# -----------------------------------------------------------
# Base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Model
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

# Configuration
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# 2. DATA

In [3]:
df = pd.read_feather("input/applications_traintest_feather")
pos = pd.read_feather("input/poscashbalance_agg_feather")
bb = pd.read_feather("input/bureau_bureaubalance_agg_feather")
cc = pd.read_feather("input/cc_feather")
ins = pd.read_feather("input/installments_payments_agg_feather")
prev = pd.read_feather("input/previous_applications_agg_feather")

print(df.shape, pos.shape, bb.shape, cc.shape, ins.shape, prev.shape)

for i in [pos, bb, cc, ins, prev]:
    df = pd.merge(df, i , how = "left", on = "SK_ID_CURR")
    
print(df.shape)

del pos, bb, ins, cc, prev

(356249, 145) (337252, 23) (305811, 456) (103558, 117) (339587, 49) (338857, 378)
(356249, 1163)


# 3. TRAIN-TEST SPLIT

In [4]:
# Train-Test Split
df.columns = list(map(lambda x: str(x).replace(" ", "_").replace("-", "_").replace("_/_", "_").upper(), df.columns))

df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [6]:
df.to_csv("df.csv")

In [14]:
train = df[df.TARGET.isnull() == False]
test = df[df.TARGET.isnull()]

x_train = train.drop(["TARGET", "SK_ID_CURR"], axis = 1)
x_test = test.drop(["TARGET", "SK_ID_CURR"], axis = 1)
y_train = train.TARGET

# 4. MODEL

In [17]:
# LightGBM parameters found by Bayesian optimization
clf = LGBMClassifier(
    nthread=4,
    n_estimators=11500,
    learning_rate=0.02,
    num_leaves=34,
    colsample_bytree=0.949,
    subsample=0.8,
    max_depth=8,
    reg_alpha=0.041,
    reg_lambda=0.07,
    min_split_gain=0.02,
    min_child_weight=39,
    silent=-1,
    verbose=-1, )

clf.fit(x_train, y_train, eval_set=[(x_train, y_train)],
        eval_metric='auc', verbose=200)


[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
[200]	training's auc: 0.797076	training's binary_logloss: 0.234734
[400]	training's auc: 0.819584	training's binary_logloss: 0.225428
[600]	training's auc: 0.834733	training's binary_logloss: 0.21924
[800]	training's auc: 0.846919	training's binary_logloss: 0.21424
[1000]	training's auc: 0.857601	training's binary_logloss: 0.209715
[1200]	training's auc: 0.86689	training's binary_logloss: 0.205642
[1400]	training's auc: 0.875547	training's binary_logloss: 0.201722
[1600]	training's auc: 0.883358	training's binary_logloss: 0.198045
[1800]	training's auc: 0.890851	training's binary_logloss: 0.194439
[2000]	training's auc: 0.897459	training's binary_logloss: 0.191084
[2200]	training's auc: 0.903979	training's binary_logloss: 0.187685
[2400]	training's auc: 0.909949	training's binary_logloss: 0.184463
[2600]	training's auc: 0.915468	training's binary_logloss: 0.181372
[2800]	tra

LGBMClassifier(colsample_bytree=0.949, learning_rate=0.02, max_depth=8,
               min_child_weight=39, min_split_gain=0.02, n_estimators=11500,
               nthread=4, num_leaves=34, reg_alpha=0.041, reg_lambda=0.07,
               silent=-1, subsample=0.8, verbose=-1)

# 5. Predict

In [18]:
submission = pd.DataFrame({
    "SK_ID_CURR": test.SK_ID_CURR,
    "TARGET": clf.predict_proba(x_test)[:,1]
})
submission.to_csv("predict.csv", index = None)